In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import librosa
import librosa.display

#from pydub import AudioSegment
#from pydub.silence import split_on_silence

from scipy.signal import find_peaks 

import os

In [2]:
path = os.getcwd()

In [3]:
metadata = pd.read_csv("metadata_compiled.csv")

In [4]:
files = metadata[metadata["cough_detected"]>=0.5]["uuid"].tolist()

In [5]:
def moving_average(a, n=100):
    ret = np.nancumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    ret = ret[n - 1:] / n
    zero_arr = [0]*(n-1)
    return np.append(zero_arr, ret)

In [6]:
for file in files:
    sig, sr = librosa.load(path+"\\public_dataset\\"+file+".wav", sr=16000)
    
    #원본
    fig=plt.figure(figsize=(4,4))
    librosa.display.waveplot(sig, sr, alpha=0.5)
    plt.savefig('./original_plot/'+file+'.png')
    plt.close(fig)
    
    fp = find_peaks(list(sig), height=0.2)[0] #높이가 최소 0.2 이상인 값을 찾음

    fp_diff = np.diff(fp)>3000 #다음 기침은 최소 0.2초 이후에 한다 가정
    fp_diff = np.append(fp_diff, False) #차분 값의 마지막에 False 값 추가
    if sum(fp_diff)==0: fp_index = 0
    else: fp_index = list(fp[fp_diff]) #True인 값 추출 (차이가 커지기 전 인덱스 추출)
    
    #한주기
    if fp_index==0:
        fig=plt.figure(figsize=(4,4))
        librosa.display.waveplot(sig, sr, alpha=0.5)
        plt.savefig('./one_plot/'+file+'.png')
        plt.close(fig)
        
        yt,idx = librosa.effects.trim(sig, top_db = 10)
        
        fig=plt.figure(figsize=(4,4))
        librosa.display.waveplot(yt, sr, alpha=0.5)
        plt.savefig('./cut_plot/'+file+'.png')
        plt.close(fig)
        
        mv = moving_average(yt, 10)

        if sum(mv>0.2)!=0:
            mv_idx = list(mv>0.2).index(True)
            
            if len(yt[(mv_idx-100):])<1600: continue
            else:
                fig=plt.figure(figsize=(4,4))
                librosa.display.waveplot(yt[(mv_idx-100):], sr, alpha=0.5)
                plt.savefig('./mv_plot/'+file+'.png')
                plt.close(fig)
                
    else:
        fig=plt.figure(figsize=(4,4))
        librosa.display.waveplot(sig[:fp_index[0]], sr, alpha=0.5)
        plt.savefig('./one_plot/'+file+'.png')
        plt.close(fig)
        
        #cutting
        yt,idx = librosa.effects.trim(sig[:fp_index[0]], top_db = 10)
        
        fig=plt.figure(figsize=(4,4))
        librosa.display.waveplot(yt, sr, alpha=0.5)
        plt.savefig('./cut_plot/'+file+'.png')
        plt.close(fig)
        
        mv = moving_average(yt, 10)
        
        if sum(mv>0.2)!=0:

            mv_idx = list(mv>0.2).index(True)

            if len(yt[(mv_idx-100):])<1600: continue
            else:
                fig=plt.figure(figsize=(4,4))
                librosa.display.waveplot(yt[(mv_idx-100):], sr, alpha=0.5)
                plt.savefig('./mv_plot/'+file+'.png')
                plt.close(fig)